In [1]:
import logging
import constants
import boto3
from botocore.exceptions import ClientError
import os

loading dot env...


In [2]:
# use third party object storage to create an S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=constants.AWS_S3_ENDPOINT,
    aws_access_key_id=constants.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=constants.AWS_SECRET_ACCESS_KEY,
)

bucket = constants.AWS_S3_BUCKET

In [11]:
#list contents of bucket 
response = s3_client.list_objects_v2(Bucket=bucket)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No objects found in the bucket.")

2023-06-01.csv
2023-06-02.csv
STAC_LiDAR/
STAC_LiDAR/PointClouds/
_$folder$
geotest/
geotest/_$folder$
geotest/cog/082E01_rgb.tif
geotest/cog/_$folder$
geotest/cog/lzw2_082E01_rgb.tif
geotest/cog/lzw_082E01_rgb.tif
geotest/veg_comp_layer_r1_poly.parquet


In [4]:
#Create Folder Like Structure(empty objects) for STAC Lidar base area and point clouds, if they do no exist already  .laz files will go into point clouds object
base_obj="STAC_LiDAR/"
pc_obj="STAC_LiDAR/PointClouds/"
obj_list=[base_obj,pc_obj]

for item in obj_list:
    response = s3_client.list_objects_v2(Bucket=bucket, Prefix=item)
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'] == item:
                print(f"The object {item}' in bucket '{bucket}' exists.")
                break
        else:
            print(f"The object {item}' in bucket '{bucket}' dose not exist, adding object")
            s3_client.put_object(Bucket=bucket, Key=item)


# s3_client.put_object(Bucket=bucket, Key=pc_obj)

The object STAC_LiDAR/' in bucket 'rczimv' exists.
The object STAC_LiDAR/PointClouds/' in bucket 'rczimv' exists.


In [12]:
# specify the folder path (in the current working directory) where objects are/will be
# if the folder doesn't exist, create it
save_path = (r'/home/cfolkers/STAC_LiDAR/PointClouds')
if not os.path.exists(save_path):
    os.makedirs(save_path)


In [16]:
# put .laz files into your object storage "STAC_LiDAR/PointClouds/"
laz_list=os.listdir(save_path)

for obj in laz_list:
    obj_load=os.path.join(save_path,obj)
    obj_key=pc_obj+obj
    s3_client.upload_file(obj_load, Bucket=bucket, Key=obj_key)
    
print(f"Objects uploaded to folder '{pc_obj}' in bucket '{bucket}'.")


Objects uploaded to folder 'STAC_LiDAR/PointClouds/' in bucket 'rczimv'.


In [17]:
#list .laz objects in bucket
object_key="STAC_LiDAR/PointClouds/"
laz_objects=[]

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=object_key, StartAfter=object_key)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        laz_objects.append(obj['Key'])
        print(obj['Key'])
        print(f"Object Size {obj['Size']}")
else:
    print("No objects found in the bucket.")

STAC_LiDAR/PointClouds/bc_092o018_3_2_4_xyes_12_utm10_2018.laz
Object Size 23236093
STAC_LiDAR/PointClouds/bc_092o018_3_4_2_xyes_12_utm10_2018.laz
Object Size 140355729
STAC_LiDAR/PointClouds/bc_092o018_3_4_4_xyes_12_utm10_2018.laz
Object Size 50122462
STAC_LiDAR/PointClouds/bc_092o018_4_1_3_xyes_12_utm10_2018.laz
Object Size 95552259
STAC_LiDAR/PointClouds/bc_092o018_4_1_4_xyes_12_utm10_2018.laz
Object Size 336226672
STAC_LiDAR/PointClouds/bc_092o018_4_3_1_xyes_12_utm10_2018.laz
Object Size 315611463
STAC_LiDAR/PointClouds/bc_092o018_4_3_2_xyes_12_utm10_2018.laz
Object Size 354790466
STAC_LiDAR/PointClouds/bc_092o018_4_3_3_xyes_12_utm10_2018.laz
Object Size 312537985
STAC_LiDAR/PointClouds/bc_092o018_4_3_4_xyes_12_utm10_2018.laz
Object Size 332078905
